In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from looplm.config.manager import ConfigManager
from looplm.chat.session import ChatSession
from looplm.conversation.handler import ConversationHandler
from looplm.commands import CommandManager
from rich.console import Console

### Configuration Management

In [1]:
from looplm.config.manager import ConfigManager
from looplm.config.providers import ProviderType

# Initialize configuration manager
config_manager = ConfigManager()
# config_manager.load_environment("bedrock")

# Check if any providers are configured
providers = config_manager.get_configured_providers()
if not providers:
    print("No providers configured")
else:
    print(f"Configured providers: {list(providers.keys())}")

# Get default provider
default_provider, default_model = config_manager.get_default_provider()
print(f"Default: {default_provider.value if default_provider else 'None'} - {default_model}")

# Configure a new provider programmatically
env_vars = {
    "OPENAI_API_KEY": "your-api-key-here"
}

config_manager.save_provider_config(
    provider=ProviderType.OPENAI,
    model_name="gpt-4o",
    env_vars=env_vars,
    is_default=True
)

### Chat Session

In [ ]:
from looplm.chat.session import ChatSession
from pathlib import Path

# Create a new chat session
session = ChatSession(
    name="My API Session",
    base_path=Path.cwd()
)

# Set a system prompt
session.set_system_prompt("You are a helpful Python programming assistant.")

# Send a message (this will call the LLM)
try:
    response = session.send_message(
        "What are Python decorators?",
        stream=False,  # Set to True for streaming
        show_tokens=True
    )
    print(f"Response: {response}")
except Exception as e:
    print(f"Error: {e}")

# Check token usage
usage = session.total_usage
print(f"Total tokens: {usage.total_tokens}, Cost: ${usage.cost:.6f}")

### Session Management and Persistence

In [ ]:
from looplm.chat.persistence import SessionManager

# Initialize session manager
session_manager = SessionManager()

# Create a new session
session = session_manager.create_session("API Test Session")

# Set system prompt
session.set_system_prompt("You are a code reviewer.")

# Send messages
session.send_message("Review this code: print('hello world')", stream=False)

# Save the session
session_manager.save_session(session)
print(f"Session saved with ID: {session.id}")

# List all sessions
sessions = session_manager.get_session_list()
for s in sessions:
    print(f"ID: {s['id'][:8]}, Name: {s['name']}, Messages: {s['message_count']}")

# Load a session
loaded_session = session_manager.load_session(session.id)
if loaded_session:
    print(f"Loaded session: {loaded_session.name}")

### Command Processing

In [ ]:
from looplm.commands import CommandManager
import asyncio

# Initialize command manager
command_manager = CommandManager()

# Process text with commands
async def process_with_commands():
    text = """
    Analyze this file: @file(example.py)
    And also check this folder: @folder(src/)
    Here's the output of a command: $(ls -la)
    """

    processed_text, image_metadata = await command_manager.process_text(text)
    print("Processed text:")
    print(processed_text)

    if image_metadata:
        print("Image metadata:")
        for img in image_metadata:
            print(img)

# Run the async function
asyncio.run(process_with_commands())

# Or use the synchronous wrapper
processed_text, image_metadata = command_manager.process_text_sync(text)

### Register Custom Command

In [ ]:
from looplm.commands.processor import CommandProcessor, ProcessingResult
from typing import List

class DateProcessor(CommandProcessor):
    @property
    def name(self) -> str:
        return "date"

    @property
    def description(self) -> str:
        return "Get current date and time"

    def validate(self, arg: str) -> bool:
        return True  # Accept any format

    async def process(self, arg: str) -> ProcessingResult:
        import datetime
        now = datetime.datetime.now()

        if not arg.strip():
            formatted = now.strftime("%Y-%m-%d %H:%M:%S")
        else:
            try:
                formatted = now.strftime(arg)
            except ValueError:
                return ProcessingResult(
                    content="",
                    error=f"Invalid date format: {arg}"
                )

        return ProcessingResult(
            content=f"<@date>\n{formatted}\n</@date>"
        )

    def get_completions(self, text: str) -> List[str]:
        return ["%Y-%m-%d", "%Y-%m-%d %H:%M:%S", "%H:%M:%S"]

# Register the custom command
command_manager.register_command(DateProcessor)

# Now you can use @date in your text
text_with_date = "Current time: @date(%H:%M:%S)"
result, _ = command_manager.process_text_sync(text_with_date)
print(result)

### Simple Conversation Handler

In [ ]:
from looplm.conversation.handler import ConversationHandler
from rich.console import Console

# Initialize console and handler
console = Console()
handler = ConversationHandler(console)

# Handle a prompt (processes commands and sends to LLM)
try:
    handler.handle_prompt(
        "Explain this code: @file(script.py)",
        provider="openai",  # Optional provider override
        model="gpt-4o"      # Optional model override
    )
except Exception as e:
    print(f"Error: {e}")

### Debug Mode

In [ ]:
# Use debug mode to see processed commands without sending to LLM
handler = ConversationHandler(console, debug=True)
handler.handle_prompt("Check this: @file(config.json) and $(pwd)")
# This will show the processed content without calling the LLM

### QA Bot

In [ ]:
from looplm.config.manager import ConfigManager
from looplm.chat.session import ChatSession

def create_qa_bot():
    # Check configuration
    config_manager = ConfigManager()
    if not config_manager.get_configured_providers():
        print("Please configure a provider first")
        return None

    # Create session
    session = ChatSession(name="QA Bot")
    session.set_system_prompt("You are a helpful assistant that gives concise answers.")

    return session

def ask_question(session, question):
    try:
        response = session.send_message(question, stream=False)
        return response
    except Exception as e:
        return f"Error: {e}"

# Usage
bot = create_qa_bot()
if bot:
    answer = ask_question(bot, "What is Python?")
    print(answer)

    # Check usage
    print(f"Tokens used: {bot.total_usage.total_tokens}")

### File Analysis

In [ ]:

from looplm.commands import CommandManager
from looplm.conversation.handler import ConversationHandler
from rich.console import Console
import asyncio

class FileAnalyzer:
    def __init__(self):
        self.console = Console()
        self.handler = ConversationHandler(self.console)
        self.command_manager = CommandManager()

    async def analyze_file(self, file_path, analysis_type="general"):
        """Analyze a file with specific instructions"""

        prompts = {
            "general": f"Analyze this file and provide a summary: @file({file_path})",
            "security": f"Review this file for security issues: @file({file_path})",
            "performance": f"Analyze this code for performance issues: @file({file_path})",
            "documentation": f"Review the documentation in this file: @file({file_path})"
        }

        prompt = prompts.get(analysis_type, prompts["general"])

        try:
            # Process commands first to see what will be sent
            processed_text, images = await self.command_manager.process_text(prompt)
            print("="*50)
            print("PROCESSED PROMPT:")
            print("="*50)
            print(processed_text[:500] + "..." if len(processed_text) > 500 else processed_text)
            print("="*50)

            # Send to LLM
            self.handler.handle_prompt(prompt)

        except Exception as e:
            print(f"Error analyzing file: {e}")

    def analyze_file_sync(self, file_path, analysis_type="general"):
        """Synchronous wrapper"""
        asyncio.run(self.analyze_file(file_path, analysis_type))

# Usage
analyzer = FileAnalyzer()
analyzer.analyze_file_sync("example.py", "security")

### Multi-session Chat

In [ ]:
from looplm.chat.persistence import SessionManager
from looplm.chat.session import ChatSession
from typing import Dict, Optional

class ChatManager:
    def __init__(self):
        self.session_manager = SessionManager()
        self.sessions: Dict[str, ChatSession] = {}

    def create_session(self, name: str, system_prompt: str = None) -> str:
        """Create a new chat session"""
        session = self.session_manager.create_session(name)

        if system_prompt:
            session.set_system_prompt(system_prompt)

        self.sessions[session.id] = session
        return session.id

    def send_message(self, session_id: str, message: str) -> str:
        """Send message to a specific session"""
        if session_id not in self.sessions:
            raise ValueError(f"Session {session_id} not found")

        session = self.sessions[session_id]
        return session.send_message(message, stream=False)

    def save_session(self, session_id: str) -> bool:
        """Save a session to disk"""
        if session_id not in self.sessions:
            return False

        session = self.sessions[session_id]
        return self.session_manager.save_session(session)

    def load_session(self, session_id: str) -> bool:
        """Load a session from disk"""
        session = self.session_manager.load_session(session_id)
        if session:
            self.sessions[session.id] = session
            return True
        return False

    def list_sessions(self) -> list:
        """List all saved sessions"""
        return self.session_manager.get_session_list()

    def get_session_usage(self, session_id: str) -> dict:
        """Get token usage for a session"""
        if session_id not in self.sessions:
            return {}

        session = self.sessions[session_id]
        return session.total_usage.to_dict()

# Usage
chat_manager = ChatManager()

# Create specialized sessions
code_session = chat_manager.create_session(
    "Code Helper",
    "You are an expert programmer who helps with code review and debugging."
)

writing_session = chat_manager.create_session(
    "Writing Assistant",
    "You are a professional editor who helps improve writing quality."
)

# Use sessions
code_response = chat_manager.send_message(
    code_session,
    "Review this function: @file(utils.py)"
)

writing_response = chat_manager.send_message(
    writing_session,
    "Improve this text: @file(draft.md)"
)

# Save sessions
chat_manager.save_session(code_session)
chat_manager.save_session(writing_session)

# Check usage
print("Code session usage:", chat_manager.get_session_usage(code_session))
print("Writing session usage:", chat_manager.get_session_usage(writing_session))

### Batch Processing

In [ ]:

from looplm.commands import CommandManager
from looplm.conversation.handler import ConversationHandler
from rich.console import Console
from pathlib import Path
import asyncio

class BatchProcessor:
    def __init__(self):
        self.console = Console()
        self.handler = ConversationHandler(self.console)
        self.command_manager = CommandManager()
        self.results = []

    async def process_files(self, file_patterns: list, prompt_template: str):
        """Process multiple files with a template prompt"""

        files = []
        for pattern in file_patterns:
            files.extend(Path.cwd().glob(pattern))

        for file_path in files:
            print(f"\nProcessing: {file_path}")
            print("-" * 50)

            # Create prompt for this file
            prompt = prompt_template.format(file=str(file_path))

            try:
                # Process with command manager first
                processed_text, images = await self.command_manager.process_text(prompt)

                # Store result
                result = {
                    'file': str(file_path),
                    'prompt': prompt,
                    'processed_text': processed_text,
                    'success': True
                }

                # Send to LLM (in real scenario)
                # self.handler.handle_prompt(prompt)

                self.results.append(result)

            except Exception as e:
                print(f"Error processing {file_path}: {e}")
                self.results.append({
                    'file': str(file_path),
                    'error': str(e),
                    'success': False
                })

    def generate_report(self) -> str:
        """Generate a summary report"""
        successful = sum(1 for r in self.results if r.get('success', False))
        total = len(self.results)

        report = f"Batch Processing Report\n"
        report += f"=====================\n"
        report += f"Total files: {total}\n"
        report += f"Successful: {successful}\n"
        report += f"Failed: {total - successful}\n\n"

        for result in self.results:
            if result.get('success'):
                report += f"✓ {result['file']}\n"
            else:
                report += f"✗ {result['file']} - {result.get('error', 'Unknown error')}\n"

        return report

# Usage
async def main():
    processor = BatchProcessor()

    # Process all Python files
    await processor.process_files(
        ["*.py", "src/*.py"],
        "Analyze this Python file for code quality: @file({file})"
    )

    print(processor.generate_report())

# Run batch processing
asyncio.run(main())

In [3]:
from looplm.config.manager import ConfigManager
from looplm.chat.session import ChatSession
from looplm.chat.persistence import SessionManager
from looplm.utils.prompts import PromptsManager
from pathlib import Path

# Load your keys
config_manager = ConfigManager()
config_manager.load_environment("bedrock")

# Create session manager and new session
session_manager = SessionManager()
session = session_manager.create_session("My Chat Session")

# Set system prompt (optional)
prompts_manager = PromptsManager()
default_prompt = prompts_manager.get_prompt("default")
session.set_system_prompt(default_prompt)

# Send messages
response1 = session.send_message("Hello, how are you?", stream=True)
print(f"Response: {response1}")

response2 = session.send_message("Tell me about Python", stream=True)
print(f"Response: {response2}")

# Save session
session_manager.save_session(session)

08:50 

Assistant ▣

Output()

Response: I am doing well, thank you for asking! How are you today?



08:50 

Assistant ▣

Output()

Response: Okay! Here's a breakdown of Python, covering its key aspects, history, uses, and why it's so popular:

**What is Python?**

Python is a high-level, general-purpose programming language.  That means:

*   **High-level:** It's designed to be easy for humans to read and write, abstracting away many of the low-level details of computer hardware.
*   **General-purpose:** It can be used for a wide variety of tasks, from web development and data science to scripting and automation.

**Key Features and Characteristics:**

*   **Readability:** Python emphasizes code readability.  Its syntax is clean and uses indentation to define code blocks, making it easier to understand and maintain.  This "executable pseudocode" approach is a major draw for beginners.
*   **Interpreted:** Python code is executed line by line by an interpreter, rather than being compiled into machine code beforehand. This makes development faster and more flexible. (However, there are also ways to compile Python co

True

In [3]:
# File inclusion
prompt_with_file = "Explain this code: @file(script.py)"
response = session.send_message(prompt_with_file)

# Folder analysis
prompt_with_folder = "Analyze this project structure: @folder(./my_project)"
response = session.send_message(prompt_with_folder)

# Shell command output
prompt_with_shell = "Explain this process list: $(ps aux | head -10)"
response = session.send_message(prompt_with_shell)

# GitHub repository
prompt_with_github = "Review this repo: @github(https://github.com/user/repo)"
response = session.send_message(prompt_with_github)

Exception: Error sending message: Command processing failed:
@file error: Invalid argument for @file: script.py

In [4]:
# Using specific provider and model
session.set_model("claude-3-5-sonnet-20240620", "anthropic")

# Or create session with specific provider
from looplm.chat.control import CommandHandler

handler = CommandHandler(provider="anthropic", model="claude-3-5-sonnet-20240620")
handler.start_session()  # This starts interactive mode

# For programmatic use, access the session
session = handler.session_manager.active_session
response = session.send_message("Your prompt here")


{'default_model': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'models': ['anthropic.claude-3-5-sonnet-20240620-v1:0'],
 'env_vars': ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_REGION_NAME']}

In [5]:

# Debug mode - shows processed content without sending to LLM
response = session.send_message(
"Check this file: @file(config.py)",
debug=True
)

In [7]:
# List existing sessions
sessions = session_manager.get_session_list()
print(sessions)

# Load existing session
if sessions:
    session_id = sessions[0]['id']
    loaded_session = session_manager.load_session(session_id)
    if loaded_session:
        response = loaded_session.send_message("Continue our conversation")

# Clear session history
session.clear_history()

# Clear last N messages
session.clear_last_messages(2)  # Clear last 2 messages

# View token usage
print(session.total_usage.to_dict())

0.0005639999999999999

In [3]:
from looplm.config.manager import ConfigManager
from looplm.chat.session import ChatSession
from looplm.chat.persistence import SessionManager
from looplm.utils.prompts import PromptsManager

def main():
    # Setup
    config_manager = ConfigManager()
    config_manager.load_environment("bedrock")

    session_manager = SessionManager()
    session = session_manager.create_session("API Test Session")

    # Set system prompt
    prompts_manager = PromptsManager()
    session.set_system_prompt("You are a helpful coding assistant.")

    try:
        # Send messages
        response1 = session.send_message(
            "Hello! Can you help me with Python?",
            stream=False  # Set to True for streaming
        )
        print(f"Assistant: {response1}")

        # Use commands
        response2 = session.send_message(
            "What's in my current directory? $(ls -la)",
            stream=False
        )
        print(f"Assistant: {response2}")

        # Show usage stats
        usage = session.total_usage.to_dict()
        print(f"Tokens used: {usage['total_tokens']}, Cost: ${usage['cost']:.6f}")

        # Save session
        session_manager.save_session(session)
        print(f"Session saved as: {session.name}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


08:37 

Assistant ▣

I'd love to!  To best help you, please tell me:                                                                    

 • What are you trying to do?  Describe the task you're trying to accomplish. The more detail you provide, the     
   better I can understand your needs.                                                                             
 • What code have you written so far?  Please share any code you've already attempted.  This helps me understand   
   your current approach and identify any potential issues. Make sure to format your code properly using backticks 
   (```) so it's easy to read.                                                                                     
 • What errors are you encountering? If you're getting error messages, please provide the full error message,      
   including the traceback. This is crucial for debugging.                                                         
 • What are your specific questions?  What are you unsure about? Are you stuck on a particular concept or line of  
   code?                                                                                                           

For example, you could say:                                                                                        

"I'm trying to write a Python function that reads a CSV file and calculates the average value of a specific column.
I've written some code to open the file, but I'm not sure how to read the data and perform the calculation. I'm    
also getting a FileNotFoundError. Here's my code:"                                                                 

                                                                                                                   
 import csv                                                                                                        
                                                                                                                   
 def calculate_average(filename, column_name):                                                                     
     try:                                                                                                          
         with open(filename, 'r') as file:                                                                         
             reader = csv.reader(file)                                                                             
             # I'm not sure what to do here to read the data and calculate the average                             
             return None                                                                                           
     except FileNotFoundError:                                                                                     
         print("Error: File not found.")                                                                           
         return None                                                                                               
                                                                                                                   
 # Example usage                                                                                                   
 filename = "my_data.csv"                                                                                          
 column_name = "Value"                                                                                             
 average = calculate_average(filename, column_name)                                                                
                                                                                                                   
 if average is not None:                                                                                           
     print(f"The average of the '{column_name}' column is: {average}")                                             
                                                    

Assistant: I'd love to!  To best help you, please tell me:

*   **What are you trying to do?**  Describe the task you're trying to accomplish. The more detail you provide, the better I can understand your needs.
*   **What code have you written so far?**  Please share any code you've already attempted.  This helps me understand your current approach and identify any potential issues. Make sure to format your code properly using backticks (```) so it's easy to read.
*   **What errors are you encountering?** If you're getting error messages, please provide the full error message, including the traceback. This is crucial for debugging.
*   **What are your specific questions?**  What are you unsure about? Are you stuck on a particular concept or line of code?

For example, you could say:

"I'm trying to write a Python function that reads a CSV file and calculates the average value of a specific column. I've written some code to open the file, but I'm not sure how to read the data and perfo

08:37 

Assistant ▣

Okay, thanks!  It looks like your current directory contains one file:                                             

 • sample.ipynb:  This appears to be a Jupyter Notebook file.                                                      

Given that, what Python task are you trying to perform? Are you trying to interact with this notebook file in some 
way, or are you working on something unrelated? Knowing what you're trying to do will help me provide more targeted
assistance.

Assistant: Okay, thanks!  It looks like your current directory contains one file:

*   `sample.ipynb`:  This appears to be a Jupyter Notebook file.

Given that, what Python task are you trying to perform? Are you trying to interact with this notebook file in some way, or are you working on something unrelated? Knowing what you're trying to *do* will help me provide more targeted assistance.

Tokens used: 1277, Cost: $0.000308
Session saved as: API Test Session


In [28]:
part.usage

Usage(completion_tokens=35, prompt_tokens=13, total_tokens=48, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0)

In [9]:
pprint(response)

ModelResponse(id='chatcmpl-82ee0795-aa53-49f2-86ce-ad9be3e72b57', created=1749047649, model='anthropic.claude-3-5-sonnet-20240620-v1:0', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hello world', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=20, prompt_tokens=10, total_tokens=30, completion_tokens_details=None, prompt_tokens_details=None))
